In [71]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

df_train=pd.read_csv("Train.csv")

df_train=df_train.drop(columns='Unnamed: 0')
df_train_modified=df_train[['year','id arc', 'id noeud amont',
       'id noeud aval', 'etat trafic', 'etat arc', 'Temp', 'Pluie 1h',
       'Pluie 3h', 'Neige', 'Visibilité Horiz', 'cos_hour', 'sin_hour',
       'cos_day', 'sin_day', 'cos_month', 'sin_month', 'débit horaire',
       'taux occupation']]

In [72]:
df_train[df_train["id arc"]==5672.0]

,year,month,day,hour,id arc,id noeud amont,id noeud aval,etat trafic,etat arc,Temp,...,taux occupation,hour_norm,cos_hour,sin_hour,day_norm,cos_day,sin_day,month_norm,cos_month,sin_month
2,2023,2,1,23,5672,2973,2937,1,3,280.55,...,3.92444,6.283185,1.000000,-2.449294e-16,0.202683,0.979530,0.201299,1.047198,0.5,0.866025
5,2023,2,1,22,5672,2973,2937,1,3,280.55,...,1.90222,6.010003,0.962917,-2.697968e-01,0.202683,0.979530,0.201299,1.047198,0.5,0.866025
8,2023,2,1,20,5672,2973,2937,1,3,281.55,...,6.20389,5.463639,0.682553,-7.308360e-01,0.202683,0.979530,0.201299,1.047198,0.5,0.866025
11,2023,10,4,6,5672,2973,2937,1,3,282.95,...,0.64444,1.639092,-0.068242,9.976688e-01,0.810734,0.688967,0.724793,5.235988,0.5,-0.866025
14,2023,10,4,4,5672,2973,2937,1,3,283.85,...,0.32556,1.092728,0.460065,8.878852e-01,0.810734,0.688967,0.724793,5.235988,0.5,-0.866025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22370,2023,2,1,2,5672,2973,2937,1,3,279.85,...,0.59056,0.546364,0.854419,5.195840e-01,0.202683,0.979530,0.201299,1.047198,0.5,0.866025
22373,2023,8,1,4,5672,2973,2937,1,3,291.25,...,0.70500,1.092728,0.460065,8.878852e-01,0.202683,0.979530,0.201299,4.188790,-0.5,-0.866025
22376,2023,8,1,1,5672,2973,2937,1,3,291.05,...,0.74666,0.273182,0.962917,2.697968e-01,0.202683,0.979530,0.201299,4.188790,-0.5,-0.866025
22379,2023,8,1,3,5672,2973,2937,1,3,291.25,...,0.39444,0.819546,0.682553,7.308360e-01,0.202683,0.979530,0.201299,4.188790,-0.5,-0.866025


In [73]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import model_selection
from sklearn import preprocessing
from sklearn import neighbors, metrics


In [74]:
X = df_train_modified.iloc[:, :-2]
y1 = df_train_modified[["débit horaire"]]
y2 = df_train_modified[["taux occupation"]]


X_train, X_test, y1_train, y1_test = train_test_split(X, y1, test_size= 0.2, random_state=1)
X_train, X_test, y2_train, y2_test = train_test_split(X, y2, test_size= 0.2, random_state=1)



In [75]:
y_b = df_train_modified[["taux occupation","débit horaire"]]

X_train_b, X_test_b, y_train_b, y_test_b = train_test_split(X, y_b, test_size= 0.2, random_state=1)


In [76]:
X_train_b.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17916 entries, 19208 to 235
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   year              17916 non-null  int64  
 1   id arc            17916 non-null  int64  
 2   id noeud amont    17916 non-null  int64  
 3   id noeud aval     17916 non-null  int64  
 4   etat trafic       17916 non-null  int64  
 5   etat arc          17916 non-null  int64  
 6   Temp              17916 non-null  float64
 7   Pluie 1h          17916 non-null  float64
 8   Pluie 3h          17916 non-null  float64
 9   Neige             17916 non-null  float64
 10  Visibilité Horiz  17916 non-null  float64
 11  cos_hour          17916 non-null  float64
 12  sin_hour          17916 non-null  float64
 13  cos_day           17916 non-null  float64
 14  sin_day           17916 non-null  float64
 15  cos_month         17916 non-null  float64
 16  sin_month         17916 non-null  float64
d

In [77]:
params_rf = {"max_depth": [20],
                "min_samples_split": [10],
                "max_leaf_nodes": [175],
                "min_samples_leaf": [5],
                "n_estimators": [250],
                 "max_features": ["sqrt"],
                }

rf = RandomForestRegressor(random_state=1)

rf_cv = GridSearchCV(rf, params_rf, cv = 3, n_jobs = -1, verbose = False)

rf_cv.fit(X_train.values, np.ravel(y1_train))

rf = rf_cv.best_estimator_


neigh=KNeighborsRegressor()
neigh.fit(X_train.values, np.ravel(y1_train))


# Fixer les valeurs des hyperparamètres à tester
param_grid = {'n_neighbors':[2,3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]}

# Choisir un score à optimiser, ici l'accuracy (proportion de prédictions correctes)
score = 'accuracy'

std_scale = preprocessing.StandardScaler().fit(X_train)
X_train_std = std_scale.transform(X_train)
X_test_std = std_scale.transform(X_test)

# Créer un classifieur kNN avec recherche d'hyperparamètre par validation croisée
clf = model_selection.GridSearchCV(
    neighbors.KNeighborsClassifier(), # un classifieur kNN
    param_grid,     # hyperparamètres à tester
    cv=5,           # nombre de folds de validation croisée
    scoring=score   # score à optimiser
)

# Optimiser ce classifieur sur le jeu d'entraînement
lab = preprocessing.LabelEncoder()
y1_transformed = lab.fit_transform(y2_train)
clf.fit(X_train_std, np.ravel(y1_transformed))

# Afficher le(s) hyperparamètre(s) optimaux
print("Meilleur(s) hyperparamètre(s) sur le jeu d'entraînement:")
print(clf.best_params_)
opti_neigh=KNeighborsRegressor(n_neighbors=clf.best_params_["n_neighbors"])
opti_neigh.fit(X_train_std, np.ravel(y1_train))



/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/neighbors/_classification.py:230: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be 

Meilleur(s) hyperparamètre(s) sur le jeu d'entraînement:
{'n_neighbors': 8}


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/neighbors/_classification.py:230: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


KNeighborsRegressor(n_neighbors=8)

In [78]:
print(rf.score(X_test,y1_test),metrics.mean_squared_error(y1_test, rf.predict(X_test)))
print(neigh.score(X_test,y1_test),metrics.mean_squared_error(y1_test, neigh.predict(X_test)))
print(opti_neigh.score(std_scale.transform(X_test),y1_test),metrics.mean_squared_error(y1_test, opti_neigh.predict(std_scale.transform(X_test))))

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/base.py:434: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/base.py:434: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/base.py:434: UserWarning: X has feature names, but KNeighborsRegressor was fitted without feature names
  warnings.warn(


0.9007991364043542 14953.77299497693


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/base.py:434: UserWarning: X has feature names, but KNeighborsRegressor was fitted without feature names
  warnings.warn(


0.7519338465135064 37394.07916945747
0.8562995045653383 21661.75283266354


In [79]:
params_rf = {"max_depth": [20],
                "min_samples_split": [10],
                "max_leaf_nodes": [175],
                "min_samples_leaf": [5],
                "n_estimators": [250],
                 "max_features": ["sqrt"],
                }

rf = RandomForestRegressor(random_state=1)

rf_cv = GridSearchCV(rf, params_rf, cv = 3, n_jobs = -1, verbose = False)

rf_cv.fit(X_train.values, np.ravel(y2_train))

rf = rf_cv.best_estimator_

neigh=KNeighborsRegressor()
neigh.fit(X_train.values, np.ravel(y2_train))

# Fixer les valeurs des hyperparamètres à tester
param_grid = {'n_neighbors':[2,3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]}

# Choisir un score à optimiser, ici l'accuracy (proportion de prédictions correctes)
score = 'accuracy'

std_scale = preprocessing.StandardScaler().fit(X_train)
X_train_std = std_scale.transform(X_train)
X_test_std = std_scale.transform(X_test)

# Créer un classifieur kNN avec recherche d'hyperparamètre par validation croisée
clf = model_selection.GridSearchCV(
    neighbors.KNeighborsClassifier(), # un classifieur kNN
    param_grid,     # hyperparamètres à tester
    cv=5,           # nombre de folds de validation croisée
    scoring=score   # score à optimiser
)

# Optimiser ce classifieur sur le jeu d'entraînement
lab = preprocessing.LabelEncoder()
y2_transformed = lab.fit_transform(y2_train)
clf.fit(X_train_std, y2_transformed)

# Afficher le(s) hyperparamètre(s) optimaux
print("Meilleur(s) hyperparamètre(s) sur le jeu d'entraînement:")
print(clf.best_params_)
opti_neigh=KNeighborsRegressor(n_neighbors=clf.best_params_["n_neighbors"])
opti_neigh.fit(X_train_std, np.ravel(y2_train))


print(rf.score(X_test,y2_test),metrics.mean_squared_error(y2_test, rf.predict(X_test)))
print(neigh.score(X_test,y2_test),metrics.mean_squared_error(y2_test, neigh.predict(X_test)))
print(opti_neigh.score(std_scale.transform(X_test),y2_test),metrics.mean_squared_error(y2_test, opti_neigh.predict(std_scale.transform(X_test))))


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/neighbors/_classification.py:230: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be 

Meilleur(s) hyperparamètre(s) sur le jeu d'entraînement:
{'n_neighbors': 8}
0.9331072291753861 7.321340305270103


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/base.py:434: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/base.py:434: UserWarning: X has feature names, but KNeighborsRegressor was fitted without feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/base.py:434: UserWarning: X has feature names, but KNeighborsRegressor was fitted without feature names
  warnings.warn(


0.6736889954347567 35.71438109568672
0.9214657427599302 8.595488208788705


In [82]:
params_rf = {"max_depth": [20],
                "min_samples_split": [10],
                "max_leaf_nodes": [175],
                "min_samples_leaf": [5],
                "n_estimators": [250],
                 "max_features": ["sqrt"],
                }

rf = RandomForestRegressor(random_state=1)

rf_cv = GridSearchCV(rf, params_rf, cv = 3, n_jobs = -1, verbose = False)

rf_cv.fit(X_train_b.values, y_train_b)

rf = rf_cv.best_estimator_

neigh=KNeighborsRegressor(n_neighbors=2)
neigh.fit(X_train_b.values, y_train_b)

# Fixer les valeurs des hyperparamètres à tester
param_grid = {'n_neighbors':[2,3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]}

# Choisir un score à optimiser, ici l'accuracy (proportion de prédictions correctes)
score = 'accuracy'

std_scale = preprocessing.StandardScaler().fit(X_train_b)
X_train_std = std_scale.transform(X_train_b)
X_test_std = std_scale.transform(X_test_b)

# Créer un classifieur kNN avec recherche d'hyperparamètre par validation croisée
clf = model_selection.GridSearchCV(
    neighbors.KNeighborsClassifier(), # un classifieur kNN
    param_grid,     # hyperparamètres à tester
    cv=5,           # nombre de folds de validation croisée
    scoring=score   # score à optimiser
)

# clf.fit(X_train_std, y_train_b)

# # Afficher le(s) hyperparamètre(s) optimaux
# print("Meilleur(s) hyperparamètre(s) sur le jeu d'entraînement:")
# print(clf.best_params_)
# opti_neigh=KNeighborsRegressor(n_neighbors=clf.best_params_["n_neighbors"])
# opti_neigh.fit(X_train_std, y_train_b)


print(rf.score(X_test_b,y_test_b),metrics.mean_squared_error(y_test_b, rf.predict(X_test_b)))
print(neigh.score(X_test_b,y_test_b),metrics.mean_squared_error(y_test_b, neigh.predict(X_test_b)))
# print(opti_neigh.score(std_scale.transform(X_test_b),y_test_b),metrics.mean_squared_error(y_test_b, opti_neigh.predict(std_scale.transform(X_test_b))))


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/base.py:434: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/base.py:434: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/base.py:434: UserWarning: X has feature names, but KNeighborsRegressor was fitted without feature names
  warnings.warn(


0.8872270353156622 7458.731050676397


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/base.py:434: UserWarning: X has feature names, but KNeighborsRegressor was fitted without feature names
  warnings.warn(


0.838614440270283 10289.834937604082
